In [ ]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
import pandas as pd
import cv2
import numpy as np
from PIL import Image

# Load the pre-trained Inception-ResNet-V2 model
base_model = tf.keras.applications.InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(224,224,3))

# Freeze the pre-trained weights
base_model.trainable = False

# Define the input shape
input_shape = (224, 224, 3)

# Define the input layer
inputs = tf.keras.layers.Input(shape=input_shape)

# Pass the input layer through the pre-trained model
features = base_model(inputs)

# Add a dense layer for multi-class classification
outputs = tf.keras.layers.Dense(units=8, activation='softmax')(features)

# Create the model
model = tf.keras.Model(inputs=inputs, outputs=outputs)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Load the image data and one-hot encoded multi-label data
# Load the data from the excel sheet into a pandas dataframe
# Load both left and right eye dataset
base_path = 'C:/Users/manik/DL-Project/DeepLearning/DL-Project'
left_df = pd.read_excel(f'{base_path}/updated_left_eye.xlsx')
right_df = pd.read_excel(f'{base_path}/updated_right_eye.xlsx')

#concating both dataframes
df = pd.concat([left_df, right_df], axis=0)


# Split the data into two arrays, one for the image paths and one for the labels
image_paths = df["Fundus"].values
labels = df.iloc[:, 3:].values

# Create a dataset from the image paths and labels
dataset = tf.data.Dataset.from_tensor_slices((image_paths, labels))


# Load the image data
images = []
for i in image_paths:
    img = Image.open(f'D:/cropped_training_images/{i}')
    images.append(np.array(img))

images = np.array(images)

# Split the data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(images, labels, test_size=0.2)

# Train the model
model.fit(x_train, y_train, batch_size=32, epochs=2, validation_data=(x_val, y_val))
